## Bresenham algorithm in Python/Cython

The code is in Cython.
To convert it to pure python, remove *%%cython -a*, *@cython.locals* and *cimport* commands, and replace *cpdef* by *def. Then, it will work fine in Python (but slower)

In [ ]:
%load_ext cython

In [ ]:
%%cython -a
from math import pi

import cython
from libc.math cimport sin,cos
from libc.stdlib cimport abs
@cython.locals(x=cython.int,y=cython.int,x2=cython.int,y2=cython.int,\
               w=cython.int,h=cython.int,steep=cython.int,i=cython.int,\
               sx=cython.int,sy=cython.int,d=cython.int,dx=cython.int,dy=cython.int)

# bresenham algorithm cropped in the window (0,0,w,h)
cpdef line(x,y,x2,y2,w,h):
    
    steep = 0
    dx = abs(x2 - x)
    dy = abs(y2 - y)
    if (x2 - x) > 0: sx = 1
    else: sx = -1
    
    if (y2 - y) > 0: sy = 1
    else: sy = -1

    if dy > dx:
        steep = 1
        x,y = y,x
        dx,dy = dy,dx
        sx,sy = sy,sx
        w,h   = h,w

    d = (2 * dy) - dx

    i = 0
    while x >= 0 and y >= 0 and x < w and y < h and i <= dx:
        if steep: print (y,x)
        else: print (x,y)

        while d >= 0:
            y = y + sy
            d = d - (2 * dx)
        
        x += sx
        d += 2 * dy
        i += 1

@cython.locals(x=cython.int,y=cython.int,angle=cython.float,w=cython.int,h=cython.int)

def rayon(x,y,angle,w,h):
    dx = cos(angle)*(w+h)
    dy = sin(angle)*(w+h)
    line(x,y,int(x+dx),int(y+dy),w,h)


In [4]:
line(0,0,10,3,100,100)

(0, 0)
(1, 0)
(2, 1)
(3, 1)
(4, 1)
(5, 2)
(6, 2)
(7, 2)
(8, 2)
(9, 3)
(10, 3)


In [5]:
rayon(5,5,3*pi/4,10,10)

(5, 5)
(4, 6)
(3, 7)
(2, 8)
(1, 9)
